# Création des Master Flats normalisés (corrigés du bias uniquement)

Ce notebook :
- Charge le `master_bias.fits`
- Pour chaque filtre dans `flats/` :
  1. Lit toutes les images `.fits`
  2. Soustrait le bias
  3. Combine les images corrigées (médiane pixel par pixel)
  4. Normalise le résultat par la moyenne
  5. Sauvegarde un `master_flat_<filtre>.fits` dans `master_flats/`

Les darks ne sont **pas utilisés** ici.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pathlib import Path

# Dossiers
flat_root = Path('/Volumes/Astro/ohp/lyon/Flat')  # à adapter si besoin
output_dir = Path('master')
output_dir.mkdir(exist_ok=True)

# Master bias
master_bias_path = Path('master_bias_lyon.fits')
if not master_bias_path.exists():
    raise FileNotFoundError('master_bias.fits introuvable !')

master_bias = fits.getdata(master_bias_path).astype(np.float32)

In [ ]:
# Parcours des sous-dossiers (un par filtre)
for subdir in sorted(flat_root.iterdir()):
    if not subdir.is_dir():
        continue

    filter_name = subdir.name
    flat_files = sorted(subdir.glob('*.fits'))
    if len(flat_files) == 0:
        print(f"Aucun fichier dans {subdir}")
        continue

    print(f"\nFiltre {filter_name} : {len(flat_files)} fichiers")
    flat_data = []

    for file in flat_files:
        try:
            with fits.open(file, ignore_missing_end=True) as hdul:
                data = None
                for hdu in hdul:
                    if hdu.data is not None:
                        data = hdu.data.astype(np.float32)
                        break
                if data is None:
                    continue

                corrected = data - master_bias
                flat_data.append(corrected)

        except Exception as e:
            print(f"Erreur lecture {file.name} : {e}")

    if not flat_data:
        print(f"Aucun flat valide pour {filter_name}, ignoré.")
        continue

    flat_stack = np.array(flat_data)
    master_flat = np.median(flat_stack, axis=0)

    mean_val = np.mean(master_flat)
    master_flat_norm = master_flat / mean_val

    output_file = output_dir / f'master_flat_{filter_name}.fits'
    fits.writeto(output_file, master_flat_norm.astype(np.float32), overwrite=True)
    print(f"Master flat normalisé sauvegardé : {output_file}")

    plt.figure(figsize=(6,5))
    plt.imshow(master_flat_norm, origin='lower', cmap='gray',
               vmin=np.percentile(master_flat_norm,5), vmax=np.percentile(master_flat_norm,95))
    plt.title(f'Master Flat {filter_name} (normalisé)')
    plt.colorbar(label='Valeur normalisée')
    plt.show()

    print(f"Moyenne : {np.mean(master_flat_norm):.3f} | σ : {np.std(master_flat_norm):.3f}")